In [21]:
from mosqlient import get_infodengue
import pandas as pd
import numpy as np
from datetime import date
import time
from aiohttp import ClientConnectionError

ufs = [
    "AC", "AL", "AP", "AM", "BA", "CE", "DF", "ES", "GO", "MA", 
    "MT", "MS", "MG", "PA", "PB", "PR", "PE", "PI", "RJ", "RN", 
    "RS", "RO", "RR", "SC", "SP", "SE", "TO"
]
years = range(2018, 2025)
arboviroses = ['dengue', 'chikungunya'] 
dfs = []

In [22]:
max_retries = 5
retry_delay = 300 

for arbovirose in arboviroses:
    for uf in ufs:

        if uf != ufs[0]:
            print('Esperando 5:30min')
            time.sleep(330)

        for year in years:

            print(f'{uf} - {arbovirose} - {str(year)}')

            start_date = f'{year}-01-01'
            if year == date.today().year:
                end_date = str(date.today())
            else:
                end_date = f'{year}-12-31'

            attempt = 0
            while attempt < max_retries:
                try:
                    # Tenta fazer a requisição
                    df_year = get_infodengue(
                        start_date=start_date,
                        end_date=end_date,
                        disease=arbovirose,
                        uf=uf
                    )
                    df_year['uf'] = uf
                    df_year['CID'] = arbovirose

                    dfs.append(df_year)
                    
                    time.sleep(20)
                    print('Próximo ano')
                    break 

                except ClientConnectionError as e:
                    print(f"Erro de conexão: {e}. Tentando novamente em 5 minutos.")
                    attempt += 1
                    if attempt < max_retries:
                        time.sleep(retry_delay) 
                    else:
                        print("Número máximo de tentativas atingido. Pulando para o próximo ano.")

                except Exception as e:
                    print(f"Ocorreu um erro: {e}. Tentando novamente em 5 minutos.")
                    attempt += 1
                    if attempt < max_retries:
                        time.sleep(retry_delay)  
                    else:
                        print("Número máximo de tentativas atingido. Pulando para o próximo ano.")

        # Salvando ao finalizar cada estado
        if dfs:
            df_uf = pd.concat(dfs, ignore_index=True)
            df_uf.to_csv('../data/df_mosqlimate_uf_chik.csv', sep=";", decimal=".", index=False)

# Salvando ao finalizar tudo
if dfs:
    df_final = pd.concat(dfs, ignore_index=True)
    df_final.to_csv('../data/df_mosqlimate_chik.csv', sep=";", decimal=".", index=False)

AC - chikungunya - 2018
Próximo ano
AC - chikungunya - 2019
Próximo ano
AC - chikungunya - 2020
Próximo ano
AC - chikungunya - 2021
Próximo ano
AC - chikungunya - 2022
Próximo ano
AC - chikungunya - 2023
Próximo ano
AC - chikungunya - 2024
Próximo ano


C:\Users\lucas\AppData\Local\Temp\ipykernel_45624\4081768529.py:58: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_uf = pd.concat(dfs, ignore_index=True)


Esperando 5:30min
AL - chikungunya - 2018
Próximo ano
AL - chikungunya - 2019
Próximo ano
AL - chikungunya - 2020
Próximo ano
AL - chikungunya - 2021
Próximo ano
AL - chikungunya - 2022
Próximo ano
AL - chikungunya - 2023
Próximo ano
AL - chikungunya - 2024
Próximo ano


C:\Users\lucas\AppData\Local\Temp\ipykernel_45624\4081768529.py:58: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_uf = pd.concat(dfs, ignore_index=True)


Esperando 5:30min
AP - chikungunya - 2018
Próximo ano
AP - chikungunya - 2019
Próximo ano
AP - chikungunya - 2020
Próximo ano
AP - chikungunya - 2021
Próximo ano
AP - chikungunya - 2022
Próximo ano
AP - chikungunya - 2023
Próximo ano
AP - chikungunya - 2024
Próximo ano


...

Esperando 5:30min
PB - chikungunya - 2018
Erro de conexão: Response status: 500. Reason: Internal Server Error. Tentando novamente em 5 minutos.
Próximo ano
PB - chikungunya - 2019
Erro de conexão: Response status: 500. Reason: Internal Server Error. Tentando novamente em 5 minutos.
Próximo ano
PB - chikungunya - 2020
Erro de conexão: Response status: 500. Reason: Internal Server Error. Tentando novamente em 5 minutos.
Próximo ano
PB - chikungunya - 2021
Erro de conexão: Response status: 500. Reason: Internal Server Error. Tentando novamente em 5 minutos.
Próximo ano
PB - chikungunya - 2022
Erro de conexão: Response status: 500. Reason: Internal Server Error. Tentando novamente em 5 minutos.
Próximo ano
PB - chikungunya - 2023
Erro de conexão: Response status: 500. Reason: Internal Server Error. Tentando novamente em 5 minutos.
Próximo ano
PB - chikungunya - 2024
Erro de conexão: Response status: 500. Reason: Internal Server Error. Tentando novamente em 5 minutos.
Erro de conexão: Resp

C:\Users\lucas\AppData\Local\Temp\ipykernel_45624\4081768529.py:58: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_uf = pd.concat(dfs, ignore_index=True)


Esperando 5:30min
PR - chikungunya - 2018
Erro de conexão: Response status: 500. Reason: Internal Server Error. Tentando novamente em 5 minutos.
Erro de conexão: Response status: 500. Reason: Internal Server Error. Tentando novamente em 5 minutos.
Próximo ano
PR - chikungunya - 2019
Erro de conexão: Response status: 500. Reason: Internal Server Error. Tentando novamente em 5 minutos.
Erro de conexão: Response status: 500. Reason: Internal Server Error. Tentando novamente em 5 minutos.
Próximo ano
PR - chikungunya - 2020
Erro de conexão: Response status: 500. Reason: Internal Server Error. Tentando novamente em 5 minutos.
Erro de conexão: Response status: 500. Reason: Internal Server Error. Tentando novamente em 5 minutos.
Próximo ano
PR - chikungunya - 2021
Erro de conexão: Response status: 500. Reason: Internal Server Error. Tentando novamente em 5 minutos.
Erro de conexão: Response status: 500. Reason: Internal Server Error. Tentando novamente em 5 minutos.
Próximo ano
PR - chikungun

C:\Users\lucas\AppData\Local\Temp\ipykernel_45624\4081768529.py:58: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_uf = pd.concat(dfs, ignore_index=True)


Esperando 5:30min
PE - chikungunya - 2018
